# Aircraft Longitudinal Mode Control - PID

## Python Control System Toolbox
If you are running this code at your local computer where the python control system toolbox is already installed skip or uncomment the following line.

In [ ]:
#pip install control

## Useful Tools

We define teh following three functions that are useful for control system design:

*   `zeta = mae4182.Mp2zeta(Mp)`: convert the overshoot into the correspondin value of damping for the prototype second order system;
* `mae4182.sgrid(zeta, wn)`: draw the line of the fixed damping and the circle of the fixed natural frequency in the complex plane;
* `M_p, t_r, t_s, t_d = mae4182.step_info(sysG)`: compute the time domain specifications according to the definition of the class.

In [ ]:
import control
import matplotlib.pyplot as plt
import numpy as np
import sys
import types

mae4182 = types.ModuleType('mae4182')
sys.modules['mae4182'] = mae4182

mae4182_code = '''
import control
import matplotlib.pyplot as plt
import numpy as np

import sys
import types

def Mp2zeta(Mp):
    c=np.log(Mp)**2 
    zeta = np.sqrt(c/(np.pi**2+c))                 
    return zeta


def sgrid(zeta,wn):
    axes = plt.gca()
    xmin, xmax = axes.get_xlim()
    ymin, ymax = axes.get_ylim()
    
    theta=np.linspace(0,2*np.pi,501)
    axes.plot(wn*np.cos(theta),wn*np.sin(theta),'k:')
    
    if zeta < 1:
        tan_theta=zeta/np.sqrt(1-zeta**2)
        
    if xmin < -tan_theta*ymax:
        axes.plot([0, -tan_theta*ymax],[0, ymax],'k:')
        axes.plot([0, -tan_theta*ymax],[0, ymin],'k:')
    else:
        axes.plot([0, xmin],[0, -xmin/tan_theta],'k:')
        axes.plot([0, xmin],[0, xmin/tan_theta],'k:')


def step_info(sysG):
    output = control.step_info(sysG, SettlingTimeThreshold = 0.05)
    M_p = output["Overshoot"]
    t_r = output["RiseTime"]
    t_s = output["SettlingTime"]
    output = control.step_info(sysG, RiseTimeLimits = (0,0.5))
    t_d = output["RiseTime"]
    return M_p, t_r, t_s, t_d

'''
exec(mae4182_code, mae4182.__dict__)




## Aircraft Longitudinal Dynamics

The transfer function of Piper Dakota from the elevator angle $\delta_e$ to the pitching angle $\theta$ is given by

In [ ]:
import control
import matplotlib.pyplot as plt
import numpy as np

sysG = control.tf(160*np.polymul([1, 2.5],[1, 0.7]), np.polymul([1, 5, 40],[1, 0.03, 0.06]))

display(sysG)

In [ ]:
wn, zeta, poles = control.damp(sysG)

The impulse response of the uncontrolled system is given by

In [ ]:
%matplotlib inline
plt.rcParams['text.usetex'] = True

t, y = control.impulse_response(sysG)

plt.figure(figsize=(8,6))
plt.plot(t,y)
plt.xlabel(r'$\delta_e$')
plt.ylabel(r'$\theta$')
plt.title('impulse response')
plt.grid()

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib.animation as animation
import matplotlib.patches as patches
import base64

#%matplotlib ipympl
%matplotlib widget

N = 500
t = np.linspace(0, 100, N)
t, y = control.impulse_response(sysG, t)


# generate animation
B_vertices = b'UUUzMFFFMzCIRDE3iEQxN1pDajhaQ2o45EGrOORBqzjlQKs45UCrOJg/cTuYP3E7iT2VPIk9lTxaO/Q8Wjv0PBY4BT0WOAU9VyHXPFch1zxBuWw8QblsPHq+hzt6voc79cBaOvXAWjrpwhQ56cIUOUnEwjhJxMI4/cRsOP3EbDhBxYI4QcWCOI3FfzmNxX851MWuO9TFrjvdxi5A3cYuQPTGaED0xmhAEsd7QBLHe0D9x31A/cd9QDXHnao1x52qRccys0XHMrM7xzK4O8cyuAPHQLkDx0C5o8atuaPGrblovky8aL5MvFu8XbxbvF28tLlovLS5aLxtrFy8baxcvI85E7yPORO8cUFku3FBZLukQai6pEGouj9CZbs/QmW7BUNpugVDabrEQ4i5xEOIuT1Exrc9RMa3ZkSGtWZEhrWFRDSxhUQ0sdREqK3URKitGkW9IhpFvSI='
B_vertices_window = b'/TwoPP08KDx8PjI3fD4yN9o/ATfaPwE3f0AVOH9AFTjNQAk5zUAJOTZAfDo2QHw67T4HPO0+Bzw='
vertices0 = np.frombuffer(base64.decodebytes(B_vertices), dtype=np.float16).reshape(-1,2)
vertices0_window = np.frombuffer(base64.decodebytes(B_vertices_window), dtype=np.float16).reshape(-1,2)

#theta = np.linspace(0, 50, 100)*np.pi/180
theta_deg = y
theta = theta_deg*np.pi/180

R = np.array([[np.cos(theta[0]), -np.sin(theta[0])], [np.sin(theta[0]), np.cos(theta[0])]])
vertices = R @ vertices0.T
vertices_window = R @ vertices0_window.T

patch = Polygon(vertices.T, facecolor = 'b')
patch_window = Polygon(vertices_window.T, facecolor = 'w')

fig = plt.figure(figsize = (8,6))
ax = plt.gca()
ax.add_patch(patch)
ax.add_patch(patch_window)
text_t = ax.text(-2,-3, f't = {t[0]:.2f}')
ax.axis('equal')
ax.axis('off')
plt.show()

def init():
    return patch, patch_window

def animate(i):
    R = np.array([[np.cos(theta[i]), -np.sin(theta[i])], [np.sin(theta[i]), np.cos(theta[i])]])
    vertices = R @ vertices0.T
    vertices_window = R @ vertices0_window.T

    patch.set_xy(vertices.T)
    patch_window.set_xy(vertices_window.T)
    text_t.set_text(f't = {t[i]:.2f}')
    return patch, patch_window

ani = animation.FuncAnimation(fig, animate, N, init_func=init, interval=10, repeat=False)


## Time-Domain Control Design

We wish to design a control system such that
* Overshoot $M_p\leq 0.1$
* Rise time $t_r\leq 1.5$
* Steady state error due to $r=t$ is less than 0.1

To satisfy the last design spec, the system type should be greater than or equal to 1. As such, we need an integral term. Here we apply PID control with

$$ C(s) = \frac{K_d s^2 + Ks +K_i}{s} = \frac{ K (T_d s^2 + s + T_i)}{s}. $$

The error constant $K_v$ is given by

$$ K_v = \lim_{s\rightarrow 0} s C(s) G(s) = K_i G(0) = 116.67 K_i. $$

Thus, the steady state error due to the ramp input is

$$ e_{ss} = \frac{1}{K_v} = \frac{1}{116.67 K_i} \leq 0.1, $$ 

which is equivalent to $K_i \geq \frac{1}{11.667} = 0.085$.
In other words, the last design spec can be satisfied if $K_i \geq 0.085$.

Next, the first two  design specifications are converted into the corresponding values of $(\zeta, \omega_n)$ as follows.

In [ ]:
Mp = 0.1
tr = 1.5

zeta = mae4182.Mp2zeta(Mp)
wn = (0.8+2.5*zeta)/tr

print(f'zeta = {zeta:.2f}, wn={wn:.2f}')

### PID Control

The characteristic equation for the closed-loop system is 
$$1+C(s)G(s)= 1 + K C_0(s) G(s)=0,$$
where $C_0(s) = \dfrac{T_d s^2 + s + T_i}{s}$.

We first fix $T_i=2$, then design the PD part of the controller.

In [ ]:
%matplotlib widget 

Ti = 2
Td = 0.2

sysC0 = control.tf([Td, 1, Ti],[1, 0])

plt.figure()
#roots, K = control.root_locus(sysC0*sysG,grid=False)
roots, K = control.root_locus(sysC0*sysG, xlim=[-6,1], ylim=[-10,10], grid=False)
axes = plt.gca()
axes.set_title('T_d='+str(Td)+', T_i='+str(Ti))
        
mae4182.sgrid(zeta, wn)

We simulate the step response and find the time-domain specs. 

In [ ]:
%matplotlib inline

K = 0.65
Kd = K*Td
Ki = K*Ti

sysC = K*sysC0
sysYR = control.feedback(sysC*sysG,1)
t, y = control.step_response(sysYR)
plt.figure()
plt.plot(t,y)
plt.xlabel('t')
plt.ylabel('y')
plt.title('step response')
plt.grid()


M_p, t_r, t_s, t_d = mae4182.step_info(sysYR)
print(f'K_i = {Ki:.2f}')
print(f'M_p = {M_p:.2f}')
print(f't_r = {t_r:.2f}')



Next, we verify the steady state error due to a ramp input.

In [ ]:
%matplotlib inline

t = np.linspace(0,20,501)
r = t

t, y = control.forced_response(sysYR, t, r)
e = r - y

fig = plt.figure(figsize = (14,6))
plt.subplot(1,2,1)
plt.plot(t, y, 'b', t, r, 'r')
plt.xlabel('t')
plt.ylabel('y')
plt.grid()
plt.subplot(1,2,2)
plt.plot(t,e)
plt.xlabel('t')
plt.ylabel('e')
plt.grid()

print(f'e_ss = {e[-1]:.2f}')

Finally, we simulate a specific case to make the pitch angle $\theta = 10^\circ$.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import matplotlib.animation as animation
import matplotlib.patches as patches
import base64

%matplotlib widget

N = 500
t = np.linspace(0, 5, N)
t, y = control.forced_response(sysYR, t, 10)

# generate animation
B_vertices = b'UUUzMFFFMzCIRDE3iEQxN1pDajhaQ2o45EGrOORBqzjlQKs45UCrOJg/cTuYP3E7iT2VPIk9lTxaO/Q8Wjv0PBY4BT0WOAU9VyHXPFch1zxBuWw8QblsPHq+hzt6voc79cBaOvXAWjrpwhQ56cIUOUnEwjhJxMI4/cRsOP3EbDhBxYI4QcWCOI3FfzmNxX851MWuO9TFrjvdxi5A3cYuQPTGaED0xmhAEsd7QBLHe0D9x31A/cd9QDXHnao1x52qRccys0XHMrM7xzK4O8cyuAPHQLkDx0C5o8atuaPGrblovky8aL5MvFu8XbxbvF28tLlovLS5aLxtrFy8baxcvI85E7yPORO8cUFku3FBZLukQai6pEGouj9CZbs/QmW7BUNpugVDabrEQ4i5xEOIuT1Exrc9RMa3ZkSGtWZEhrWFRDSxhUQ0sdREqK3URKitGkW9IhpFvSI='
B_vertices_window = b'/TwoPP08KDx8PjI3fD4yN9o/ATfaPwE3f0AVOH9AFTjNQAk5zUAJOTZAfDo2QHw67T4HPO0+Bzw='
vertices0 = np.frombuffer(base64.decodebytes(B_vertices), dtype=np.float16).reshape(-1,2)
vertices0_window = np.frombuffer(base64.decodebytes(B_vertices_window), dtype=np.float16).reshape(-1,2)

theta_deg = y
theta = theta_deg*np.pi/180

R = np.array([[np.cos(theta[0]), -np.sin(theta[0])], [np.sin(theta[0]), np.cos(theta[0])]])
vertices = R @ vertices0.T
vertices_window = R @ vertices0_window.T

patch = Polygon(vertices.T, facecolor = 'b')
patch_window = Polygon(vertices_window.T, facecolor = 'w')

fig = plt.figure(figsize = (10,8))
ax = plt.gca()
ax.add_patch(patch)
ax.add_patch(patch_window)
text_t = ax.text(-2,-3, f't = {t[0]:.2f}')
ax.axis('equal')
ax.axis('off')
plt.show()

def init():
    return patch, patch_window

def animate(i):
    R = np.array([[np.cos(theta[i]), -np.sin(theta[i])], [np.sin(theta[i]), np.cos(theta[i])]])
    vertices = R @ vertices0.T
    vertices_window = R @ vertices0_window.T

    patch.set_xy(vertices.T)
    patch_window.set_xy(vertices_window.T)
    text_t.set_text(f't = {t[i]:.2f}')
    return patch, patch_window

ani = animation.FuncAnimation(fig, animate, N, init_func=init, interval=10, repeat=False)
